<a href="https://colab.research.google.com/github/Kabongosalomon/Introduction-RL/blob/master/Session%2003/DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab setup

In [0]:
!pip install gym > /dev/null 2>&1

In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [3]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (45.2.0)


# Deep Q-Learning (DQN)


In DQN, the $Q$-function is parameterized by a neural network of parameters $\theta$. The network takes as input a state $s$ and outputs $Q(s, a, \theta)$ for all actions $a$. 

The network is trained in way that is similar to Fitted Q Iteration. At each time $T$, the agent has observed the transitions $(s_t, a_t, r_t, s_t')_{t=1}^T$, which are stored in a __replay buffer__.

In addition to the network with parameters $\theta$, DQN keeps another network with the same architecture and parameters $\tilde{\theta}$, called __target network__. 
To update the parameters $\theta$, we sample $N$ transitions from the __replay buffer__, we define the loss 

$$
L(\theta) = \sum_{i=1}^N [Q(s_i, a_i, \theta) - (r_i + \gamma\max_{a'}Q(s'_i,a', \tilde{\theta}))]^2
$$

and update 

$$
\theta \gets \theta + \eta \nabla L(\theta).
$$


Every $C$ iterations, the target network is updated as $\tilde{\theta} \gets \theta$. 

At each time $t$, DQN updates the networks as described above, selects an action according to an $\epsilon$-greedy policy, plays the action and stores the new data in the replay buffer.

In [0]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from copy import deepcopy

import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only

from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
from IPython.display import clear_output
from pathlib import Path

import random, os.path, math, glob, csv, base64, itertools, sys
from pprint import pprint

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import io
from IPython.display import HTML

## Step 1: Define the parameters

In [0]:
# Environment
env = gym.make("CartPole-v0")

# Discount factor
GAMMA = 0.99

# Batch size
BATCH_SIZE = 500
# Capacity of the replay buffer
BUFFER_CAPACITY = 10000
# Update target net every ... episodes
UPDATE_TARGET_EVERY = 20

# Initial value of epsilon
EPSILON_START = 1.0
# Parameter to decrease epsilon
DECREASE_EPSILON = 200
# Minimum value of epislon
EPSILON_MIN = 0.05

# Number of training episodes
N_EPISODES = 200

# Learning rate
LEARNING_RATE = 0.1

## Step 2: Define the replay buffer

In [0]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, state, action, reward, next_state):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = (state, action, reward, next_state)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [0]:
# create instance of replay buffer
replay_buffer = ReplayBuffer(BUFFER_CAPACITY)

## Step 3: Define the neural network architecture, objective and optimizer

In [0]:
class Net(nn.Module):
    """
    Basic neural net.
    """
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

In [0]:
# create network and target network
hidden_size = 128
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

q_net = Net(obs_size, hidden_size, n_actions)
target_net = Net(obs_size, hidden_size, n_actions)

# objective and optimizer
objective = nn.MSELoss()
optimizer = optim.Adam(params=q_net.parameters(), lr=LEARNING_RATE)

## Step 4: Implement DQN

In [0]:
#
#  Some useful functions
#

def get_q(states):
    """
    Compute Q function for a list of states
    """
    with torch.no_grad():
        states_v = torch.FloatTensor([states])
        output = q_net.forward(states_v).data.numpy()  # shape (1, len(states), dim_state)
    return output[0, :, :]  # shape (len(states), dim_state)

def dqn_policy(s):
    """
    Choose action at s using q_net
    """
    if not isinstance(s, np.ndarray):
        raise TypeError("Vector encoding expected for state variable.")
    q = get_q([s])[0]
    return q.argmax()

def eval_dqn(n_sim=5):
    """
    Monte Carlo evaluation of DQN agent
    """
    rewards = []
    copy_env = deepcopy(env)
    for sim in range(n_sim):
        done = False
        state = copy_env.reset()
        sim_rewards = 0.0
        while not done:
            action = dqn_policy(state)
            next_state, reward, done, _ = copy_env.step(action)
            sim_rewards += reward
            state = next_state
        rewards.append(sim_rewards)
    return rewards

In [0]:
def choose_action(state, epsilon):
    """
    TO BE IMPLEMENTED
    
    Return action according to an epsilon-greedy exploration policy
    """
    q_state = get_q([state])[0] # array of shape (n_actions,)
    if np.random.uniform(0, 1) < epsilon:
      action = env.action_space.sample() # random action
    else:
      action = q_state.argmax()
    return action
    

def update(state, action, reward, next_state, done):
    """
    TO BE COMPLETED
    """
    
    # add data to replay buffer
    if done:
        next_state = None
    replay_buffer.push(state, action, reward, next_state)
    
    if len(replay_buffer) < BATCH_SIZE:
        return np.inf
    
    # get batch
    # transitions = list of (state, action, reward, next_state)
    transitions = replay_buffer.sample(BATCH_SIZE)

    # 1st thing: compute Q(s_i, a_i, theta) for all (s_i, a_i)
    # in the batch
    
    # Build tensor with s_i and tensor with a_i
    batch_states = torch.FloatTensor( 
                    [ transitions[ii][0] for ii in range(BATCH_SIZE) ]
                    )
    batch_actions = torch.LongTensor(  # type is important (Long) 
                    [ transitions[ii][1] for ii in range(BATCH_SIZE) ]
                    )
    batch_rewards = torch.FloatTensor( 
                    [ transitions[ii][2] for ii in range(BATCH_SIZE) ]
                    )

    non_final_mask = torch.tensor([(transitions[ii][3] is not None) 
                                   for ii in range(BATCH_SIZE)], dtype=torch.bool)
    non_final_next_states = torch.FloatTensor(
            [transitions[ii][3]  for ii in range(BATCH_SIZE) if transitions[ii][3] is not None])
        
    next_state_values = torch.zeros(BATCH_SIZE)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    state_action_values = q_net(batch_states).gather(1, batch_actions.view(-1, 1))


    # # A simpler (and slower) way to compute all these things
    # for ii in range(BATCH_SIZE):
    #   state_ii = transitions[ii][0]
    #   action_ii = transitions[ii][1]
    #   reward_ii = transitions[ii][2]
    #   next_state_ii = transitions[ii][3]
      
    #   next_value = 0
    #   if next_state_ii is not None:
    #     next_value = # max_a Q(next_state, a) with target net
      

    #   values[ii] = # Q(state_ii)[action_ii]  with q_net
    #   targets[ii] = reward_ii + GAMMA*next_value

    

    # Compute loss - TO BE IMPLEMENTED!
    values  = state_action_values
    targets = batch_rewards + GAMMA*next_state_values
    loss = objective(values, targets.unsqueeze(1))
     
    # Optimize the model - UNCOMMENT!
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.data.numpy()

In [42]:
#
# Train
# 

EVAL_EVERY = 5
REWARD_THRESHOLD = 199

def train():
    state = env.reset()
    epsilon = EPSILON_START
    ep = 0
    total_time = 0
    while ep < N_EPISODES:
        action = choose_action(state, epsilon)

        # take action and update replay buffer and networks
        next_state, reward, done, _ = env.step(action)
        loss = update(state, action, reward, next_state, done)

        # update state
        state = next_state

        # end episode if done
        if done:
            state = env.reset()
            ep   += 1
            if ( (ep+1)% EVAL_EVERY == 0):
                rewards = eval_dqn()
                print("episode =", ep+1, ", reward = ", np.mean(rewards),
                      "loss = ", loss)
                if np.mean(rewards) >= REWARD_THRESHOLD:
                    break

            # update target network
            if ep % UPDATE_TARGET_EVERY == 0:
                target_net.load_state_dict(q_net.state_dict())
            # decrease epsilon
            epsilon = EPSILON_MIN + (EPSILON_START - EPSILON_MIN) * \
                            np.exp(-1. * ep / DECREASE_EPSILON )    

        total_time += 1

train()
rewards = eval_dqn(20)
print("")
print("mean reward after training = ", np.mean(rewards))

episode = 5 , reward =  9.6 loss =  7.595277
episode = 10 , reward =  9.6 loss =  9.408089
episode = 15 , reward =  9.8 loss =  10.62573
episode = 20 , reward =  9.8 loss =  7.853951
episode = 25 , reward =  9.6 loss =  9.134466
episode = 30 , reward =  8.6 loss =  10.380748
episode = 35 , reward =  9.4 loss =  8.81726
episode = 40 , reward =  8.8 loss =  7.2090597
episode = 45 , reward =  9.0 loss =  8.449153
episode = 50 , reward =  9.4 loss =  9.445806
episode = 55 , reward =  9.0 loss =  9.120566
episode = 60 , reward =  9.4 loss =  10.099698
episode = 65 , reward =  9.6 loss =  10.727313
episode = 70 , reward =  9.6 loss =  10.354642
episode = 75 , reward =  9.2 loss =  11.357058
episode = 80 , reward =  9.2 loss =  9.672508
episode = 85 , reward =  9.4 loss =  13.280017
episode = 90 , reward =  8.8 loss =  10.545151
episode = 95 , reward =  9.0 loss =  12.5994005
episode = 100 , reward =  9.6 loss =  14.316308
episode = 105 , reward =  9.8 loss =  11.264362
episode = 110 , reward

## Visualizing the agent

In [43]:
def show_video(directory):
    html = []
    for mp4 in Path(directory).glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay 
                      loop controls style="height: 400px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>'''.format(mp4, video_b64.decode('ascii')))
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))
    
def make_seed(seed):
    np.random.seed(seed=seed)
    torch.manual_seed(seed=seed)
  
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1013'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1013'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [44]:
env = Monitor(env, "./gym-results", force=True, video_callable=lambda episode: True)
for episode in range(1):
    done = False
    state = env.reset()
    while not done:
        action = dqn_policy(state)
        state, reward, done, info = env.step(action)
env.close()
show_video("./gym-results")